<a href="https://colab.research.google.com/github/Vaggelis-Arg/Amazon-Ecommerce-Analysis/blob/main/Amazon_Ecommerce_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import nltk
nltk.download(['punkt', 'stopwords', 'vader_lexicon', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     /home/vaggelisarg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vaggelisarg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/vaggelisarg/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/vaggelisarg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
!pip install datasets

In [8]:
from datasets import load_dataset

# List of dataset configurations and output file names
datasets_info = [
    ("raw_review_All_Beauty", "amazon_reviews_all_beauty.csv"),
    # ("raw_review_Books", "amazon_reviews_books.csv"),
    # ("raw_review_Electronics", "amazon_reviews_electronics.csv"),
    # ("raw_review_Movies_and_TV", "amazon_reviews_movies_and_tv.csv"),
    # ("raw_review_Sports_and_Outdoors", "amazon_reviews_sports_and_outdoors.csv"),
]

# Load and save each dataset
for config, filename in datasets_info:
    print(f"Loading dataset: {config}")
    dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", config, trust_remote_code=True)
    dataset["full"].to_csv(filename)
    print(f"Saved to {filename}")

Loading dataset: raw_review_All_Beauty


Creating CSV from Arrow format: 100%|██████████| 702/702 [00:17<00:00, 40.57ba/s] 

Saved to amazon_reviews_all_beauty.csv


In [9]:
!pip install contractions

In [10]:
import pandas as pd
import re
import string
import contractions

from datasets import load_dataset

# Emoticon dictionary for replacements
emoticons = {
    ":)": "happy",
    ":-)": "happy",
    ":(": "sad",
    ":-(": "sad",
    ":D": "happy",
    ":-D": "happy",
    ";)": "wink",
    ";-)": "wink"
}

# Preprocessing function
def preprocess_text(text):
    if pd.isna(text):
        return ""

    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove URLs
    text = re.sub(r"\@\w+|\#\w+", '', text)  # remove mentions and hashtags
    text = re.sub(r"\S+@\S+", '', text)  # remove emails
    text = contractions.fix(text)  # expand contractions

    # Replace emoticons
    for emoticon, word in emoticons.items():
        text = text.replace(emoticon, word)

    text = re.sub(r"[" + string.punctuation + "]", '', text)  # remove punctuation
    text = re.sub(r"\d+", '', text)  # remove numbers
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)  # reduce 3+ char repeats to 2

    text = text.strip()
    return text

# Load and clean each dataset
for config, filename in datasets_info:
    print(f"Processing: {filename}")
    df = pd.read_csv(filename)

    # Drop rows with all NaNs or missing key fields
    df.dropna(subset=["rating"], inplace=True)

    # Clean text fields
    df["cleaned_title"] = df["title"].apply(preprocess_text)
    df["cleaned_text"] = df["text"].apply(preprocess_text)

    # Save cleaned dataset
    cleaned_filename = filename.replace(".csv", "_cleaned.csv")
    df.to_csv(cleaned_filename, index=False)
    print(f"Saved cleaned data to {cleaned_filename}")


Processing: amazon_reviews_all_beauty.csv
Saved cleaned data to amazon_reviews_all_beauty_cleaned.csv
